In [1]:
import plotly.graph_objects as go
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
project_dir = '/kaggle/input/nfl-big-data-bowl-2025'

import warnings
warnings.filterwarnings("ignore")

# Helper Functions

The code in the cell below that is used to create the "animate_play" function is taken from [Nick Wan's](https://www.kaggle.com/nickwan) great notebook which can be found [here](https://www.kaggle.com/code/nickwan/animate-plays-with-plotly-real-no-lies-here).

In [2]:
colors = {
    'ARI':["#97233F","#000000","#FFB612"],
    'ATL':["#A71930","#000000","#A5ACAF"],
    'BAL':["#241773","#000000"],
    'BUF':["#00338D","#C60C30"],
    'CAR':["#0085CA","#101820","#BFC0BF"],
    'CHI':["#0B162A","#C83803"],
    'CIN':["#FB4F14","#000000"],
    'CLE':["#311D00","#FF3C00"],
    'DAL':["#003594","#041E42","#869397"],
    'DEN':["#FB4F14","#002244"],
    'DET':["#0076B6","#B0B7BC","#000000"],
    'GB' :["#203731","#FFB612"],
    'HOU':["#03202F","#A71930"],
    'IND':["#002C5F","#A2AAAD"],
    'JAX':["#101820","#D7A22A","#9F792C"],
    'KC' :["#E31837","#FFB81C"],
    'LA' :["#003594","#FFA300","#FF8200"],
    'LAC':["#0080C6","#FFC20E","#FFFFFF"],
    'LV' :["#000000","#A5ACAF"],
    'MIA':["#008E97","#FC4C02","#005778"],
    'MIN':["#4F2683","#FFC62F"],
    'NE' :["#002244","#C60C30","#B0B7BC"],
    'NO' :["#101820","#D3BC8D"],
    'NYG':["#0B2265","#A71930","#A5ACAF"],
    'NYJ':["#125740","#000000","#FFFFFF"],
    'PHI':["#004C54","#A5ACAF","#ACC0C6"],
    'PIT':["#FFB612","#101820"],
    'SEA':["#002244","#69BE28","#A5ACAF"],
    'SF' :["#AA0000","#B3995D"],
    'TB' :["#D50A0A","#FF7900","#0A0A08"],
    'TEN':["#0C2340","#4B92DB","#C8102E"],
    'WAS':["#5A1414","#FFB612"],
    'football':["#CBB67C","#663831"]
}

def preprocess_data(tracking_data, players_data):
    tracking_df = pd.merge(df,players,how="left",on = ["nflId",'displayName'])
    return tracking_df

def hex_to_rgb_array(hex_color):
    return np.array(tuple(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)))

def ColorDistance(hex1,hex2):
    if hex1 == hex2:
        return 0
    rgb1 = hex_to_rgb_array(hex1)
    rgb2 = hex_to_rgb_array(hex2)
    rm = 0.5*(rgb1[0]+rgb2[0])
    d = abs(sum((2+rm,4,3-rm)*(rgb1-rgb2)**2))**0.5
    return d

def ColorPairs(team1,team2):
    color_array_1 = colors[team1]
    color_array_2 = colors[team2]
    # If color distance is small enough then flip color order
    if ColorDistance(color_array_1[0],color_array_2[0])<500:
        return {
          team1:[color_array_1[0],color_array_1[1]],
          team2:[color_array_2[1],color_array_2[0]],
          'football':colors['football']
        }
    else:
        return {
          team1:[color_array_1[0],color_array_1[1]],
          team2:[color_array_2[0],color_array_2[1]],
          'football':colors['football']}


def animate_play(games,tracking_df,play_df,players,gameId,playId):
    selected_game_df = games.loc[games['gameId']==gameId].copy()
    selected_play_df = play_df.loc[(play_df['playId']==playId) & (play_df['gameId']==gameId)].copy()

    tracking_players_df = tracking_df.copy()
    selected_tracking_df = tracking_players_df.loc[(tracking_players_df['playId']==playId)&(tracking_players_df['gameId']==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    team_combos = list(set(selected_tracking_df['club'].unique())-set(['football']))
    color_orders = ColorPairs(team_combos[0],team_combos[1])

    line_of_scrimmage = selected_play_df['absoluteYardlineNumber'].values[0]

    if selected_tracking_df['playDirection'].values[0] == 'right':
        first_down_marker = line_of_scrimmage + selected_play_df['yardsToGo'].values[0]
    else:
        first_down_marker = line_of_scrimmage - selected_play_df['yardsToGo'].values[0]
    down = selected_play_df['down'].values[0]
    quarter = selected_play_df['quarter'].values[0]
    gameClock = selected_play_df['gameClock'].values[0]
    playDescription = selected_play_df['playDescription'].values[0]

    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    updatemenus_dict = [
      {
          "buttons": [
              {
                  "args": [None, {"frame": {"duration": 100, "redraw": False},
                              "fromcurrent": True, "transition": {"duration": 0}}],
                  "label": "Play",
                  "method": "animate"
              },
              {
                  "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                  "label": "Pause",
                  "method": "animate"
              }
          ],
          "direction": "left",
          "pad": {"r": 10, "t": 87},
          "showactive": False,
          "type": "buttons",
          "x": 0.1,
          "xanchor": "right",
          "y": 0,
          "yanchor": "top"
      }
    ]

    sliders_dict = {
      "active": 0,
      "yanchor": "top",
      "xanchor": "left",
      "currentvalue": {
          "font": {"size": 20},
          "prefix": "Frame:",
          "visible": True,
          "xanchor": "right"
      },
      "transition": {"duration": 300, "easing": "cubic-in-out"},
      "pad": {"b": 10, "t": 50},
      "len": 0.9,
      "x": 0.1,
      "y": 0,
      "steps": []
    }

    frames = []
    for frameId in sorted_frame_list:
        data = []
        data.append(
          go.Scatter(
              x=np.arange(20,110,10),
              y=[5]*len(np.arange(20,110,10)),
              mode='text',
              text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
              textfont_size = 30,
              textfont_family = "Courier New, monospace",
              textfont_color = "#ffffff",
              showlegend=False,
              hoverinfo='none'
          )
        )
        data.append(
          go.Scatter(
              x=np.arange(20,110,10),
              y=[53.5-5]*len(np.arange(20,110,10)),
              mode='text',
              text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
              textfont_size = 30,
              textfont_family = "Courier New, monospace",
              textfont_color = "#ffffff",
              showlegend=False,
              hoverinfo='none'
          )
        )
        data.append(
          go.Scatter(
              x=[line_of_scrimmage,line_of_scrimmage],
              y=[0,53.5],
              line_dash='dash',
              line_color='blue',
              showlegend=False,
              hoverinfo='none'
          )
        )
        data.append(
          go.Scatter(
              x=[first_down_marker,first_down_marker],
              y=[0,53.5],
              line_dash='dash',
              line_color='yellow',
              showlegend=False,
              hoverinfo='none'
          )
        )
        endzoneColors = {0:color_orders[selected_game_df['homeTeamAbbr'].values[0]][0],
                        110:color_orders[selected_game_df['visitorTeamAbbr'].values[0]][0]}
        for x_min in [0,110]:
            data.append(
              go.Scatter(
                  x=[x_min,x_min,x_min+10,x_min+10,x_min],
                  y=[0,53.5,53.5,0,0],
                  fill="toself",
                  fillcolor=endzoneColors[x_min],
                  mode="lines",
                  line=dict(
                      color="white",
                      width=3
                      ),
                  opacity=1,
                  showlegend= False,
                  hoverinfo ="skip"
              )
            )

        for team in selected_tracking_df['club'].unique():
            plot_df = selected_tracking_df.loc[(selected_tracking_df['club']==team) & (selected_tracking_df['frameId']==frameId)].copy()

            if team != 'football':
                hover_text_array=[]

                for nflId in plot_df['nflId'].unique():
                    selected_player_df = plot_df.loc[plot_df['nflId']==nflId]
                    nflId = int(selected_player_df['nflId'].values[0])
                    displayName = selected_player_df['displayName'].values[0]
                    s = round(selected_player_df['s'].values[0] * 2.23693629205, 3)
                    text_to_append = f"nflId:{nflId}<br>displayName:{displayName}<br>Player Speed:{s} MPH"
                    hover_text_array.append(text_to_append)

                data.append(go.Scatter(x=plot_df['x'], y=plot_df['y'],
                                      mode = 'markers',
                                      marker=go.scatter.Marker(color=color_orders[team][0],
                                                              line=go.scatter.marker.Line(width=5,
                                                                                          color=color_orders[team][1]),
                                                              size=10),
                                      name=team,hovertext=hover_text_array,hoverinfo='text'))
            else:
                data.append(go.Scatter(x=plot_df['x'], y=plot_df['y'],
                                      mode = 'markers',
                                      marker=go.scatter.Marker(
                                        color=color_orders[team][0],
                                        line=go.scatter.marker.Line(width=2,
                                                                    color=color_orders[team][1]),
                                        size=10),
                                      name=team,hoverinfo='none'))
        slider_step = {'args': [
          [frameId],
          {'frame': {'duration': 100, 'redraw': False},
            'mode': 'immediate',
            'transition': {'duration': 0}}
        ],
          'label': str(frameId),
          'method': 'animate'}
        sliders_dict['steps'].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]['data'],
        layout= layout,
        frames=frames[1:]
    )
    
    for x_min in [0,110]:
        if x_min == 0:
            angle = 270
            teamName=selected_game_df['homeTeamAbbr'].values[0]
        else:
            angle = 90
            teamName=selected_game_df['visitorTeamAbbr'].values[0]
        fig.add_annotation(
          x=x_min+5,
          y=53.5/2,
          text=teamName,
          showarrow=False,
          font=dict(
              family="Courier New, monospace",
              size=32,
              color="White"
              ),
          textangle = angle
        )
    return fig

In [3]:
player_play = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/player_play.csv")

In [4]:
tmp = player_play[['gameId', 'playId','passDefensed','routeRan']].groupby(['gameId', 'playId'])[['passDefensed','routeRan']].agg(lambda x: list(x)).reset_index()

go_route, hitch, flat, out, cross, IN, post, slant, corner, screen, angle, wheel, none = [],[],[],[],[],[],[],[],[],[],[],[],[]
pass_defensed = []

for i in range(len(tmp)):
    if "GO" in tmp['routeRan'][i]:
        go_route.append(1)
    else:
        go_route.append(0)
        
    if "HITCH" in tmp['routeRan'][i]:
        hitch.append(1)
    else:
        hitch.append(0)

    if "FLAT" in tmp['routeRan'][i]:
        flat.append(1)
    else:
        flat.append(0)

    if "OUT" in tmp['routeRan'][i]:
        out.append(1)
    else:
        out.append(0)

    if "CROSS" in tmp['routeRan'][i]:
        cross.append(1)
    else:
        cross.append(0)

    if "IN" in tmp['routeRan'][i]:
        IN.append(1)
    else:
        IN.append(0)

    if "POST" in tmp['routeRan'][i]:
        post.append(1)
    else:
        post.append(0)

    if "SLANT" in tmp['routeRan'][i]:
        slant.append(1)
    else:
        slant.append(0)

    if "CORNER" in tmp['routeRan'][i]:
        corner.append(1)
    else:
        corner.append(0)

    if "SCREEN" in tmp['routeRan'][i]:
        screen.append(1)
    else:
        screen.append(0)

    if "ANGLE" in tmp['routeRan'][i]:
        angle.append(1)
    else:
        angle.append(0)

    if "WHEEL" in tmp['routeRan'][i]:
        wheel.append(1)
    else:
        wheel.append(0)

    if 1 in tmp['passDefensed'][i]:
        pass_defensed.append(1)
    else:
        pass_defensed.append(0)


tmp['GO'] = go_route
tmp['HITCH'] = hitch
tmp['FLAT'] = flat
tmp['OUT'] = out
tmp['CROSS'] = cross
tmp['IN'] = IN
tmp['POST'] = post 
tmp['SLANT'] = slant 
tmp['CORNER'] = corner
tmp['SCREEN'] = screen
tmp['ANGLE'] = angle
tmp['WHEEL'] = wheel
tmp['pass_defensed'] = pass_defensed 

In [5]:
def bin_cont_cols(data):
    passLength_bins = []
    for i in range(len(data['passLength'])):
        if not np.isnan(data['passLength'][i]):
            if data['passLength'][i] < 0:
                passLength_bins.append("Behind_LOS")
            elif (data['passLength'][i] >= 0) and (data['passLength'][i] < 10):
                passLength_bins.append("0-10 yards")
            elif (data['passLength'][i] >= 10) and (data['passLength'][i] < 20):
                passLength_bins.append("10-20 yards")
            elif (data['passLength'][i] >= 20) and (data['passLength'][i] < 30):
                passLength_bins.append("20-30 yards")
            elif (data['passLength'][i] >= 30) and (data['passLength'][i] < 40):
                passLength_bins.append("20-40 yards")
            elif (data['passLength'][i] >= 40):
                passLength_bins.append("> 40 yards")
        else:
            passLength_bins.append("No Pass")
    
    
    yardsGained_bins = []
    for j in range(len(data['yardsGained'])):
        if data['yardsGained'][j]:
            if data['yardsGained'][j] < 0:
                yardsGained_bins.append("< 0 yards")
            elif (data['yardsGained'][j] >= 0) and (data['yardsGained'][j] < 5):
                yardsGained_bins.append("0-5 yards")
            elif (data['yardsGained'][j] >= 5) and (data['yardsGained'][j] < 10):
                yardsGained_bins.append("5-10 yards")
            elif (data['yardsGained'][j] >= 10) and (data['yardsGained'][j] < 15):
                yardsGained_bins.append("10-15 yards")
            elif (data['yardsGained'][j] >= 15) and (data['yardsGained'][j] < 20):
                yardsGained_bins.append("15-20 yards")
            elif (data['yardsGained'][j] >= 20) and (data['yardsGained'][j] < 25):
                yardsGained_bins.append("20-25 yards")
            elif (data['yardsGained'][j] >= 25) and (data['yardsGained'][j] < 30):
                yardsGained_bins.append("25-30 yards")
            elif (data['yardsGained'][j] >= 30):
                yardsGained_bins.append("> 30 yards")
        else:
            yardsGained_bins.append("N/A")
    
    penaltyYards_bins = []
    for k in range(len(data['penaltyYards'])):
        if not np.isnan(data['penaltyYards'][k]):
            if (data['penaltyYards'][k] < -5):
                penaltyYards_bins.append("< -5 yards")
            elif (data['penaltyYards'][k] >= -5) and (data['penaltyYards'][k] < 0):
                penaltyYards_bins.append("0 to -5 yards")
            elif (data['penaltyYards'][k] >= 0) and (data['penaltyYards'][k] < 5):
                penaltyYards_bins.append("0 to 5 yards")
            elif (data['penaltyYards'][k] >= 5) and (data['penaltyYards'][k] < 10):
                penaltyYards_bins.append("5 to 10 yards")
            elif (data['penaltyYards'][k] >= 10) and (data['penaltyYards'][k] < 15):
                penaltyYards_bins.append("10 to 15 yards")
            elif (data['penaltyYards'][k] >= 15) and (data['penaltyYards'][k] < 20):
                penaltyYards_bins.append("15 to 20 yards")
            elif (data['penaltyYards'][k] >= 20):
                penaltyYards_bins.append("> 20 yards")
        else:
            penaltyYards_bins.append("N/A")
            
    endzoneDist_bins = []
    for l in range(len(data['absoluteYardlineNumber'])):
        if data['absoluteYardlineNumber'][l]:
            if (data['absoluteYardlineNumber'][l] < 10):
                endzoneDist_bins.append("< 10 yards")
            elif (data['absoluteYardlineNumber'][l] >= 10) and (data['absoluteYardlineNumber'][l] < 20):
                endzoneDist_bins.append("10-20 yards")
            elif (data['absoluteYardlineNumber'][l] >= 20) and (data['absoluteYardlineNumber'][l] < 30):
                endzoneDist_bins.append("20-30 yards")
            elif (data['absoluteYardlineNumber'][l] >= 30) and (data['absoluteYardlineNumber'][l] < 40):
                endzoneDist_bins.append("30-40 yards")
            elif (data['absoluteYardlineNumber'][l] >= 40) and (data['absoluteYardlineNumber'][l] < 50):
                endzoneDist_bins.append("40-50 yards")
            elif (data['absoluteYardlineNumber'][l] >= 50) and (data['absoluteYardlineNumber'][l] < 60):
                endzoneDist_bins.append("50-60 yards")
            elif (data['absoluteYardlineNumber'][l] >= 60) and (data['absoluteYardlineNumber'][l] < 70):
                endzoneDist_bins.append("60-70 yards")
            elif (data['absoluteYardlineNumber'][l] >= 70) and (data['absoluteYardlineNumber'][l] < 80):
                endzoneDist_bins.append("70-80 yards")
            elif (data['absoluteYardlineNumber'][l] >= 80) and (data['absoluteYardlineNumber'][l] < 90):
                endzoneDist_bins.append("80-90 yards")
            elif (data['absoluteYardlineNumber'][l] >= 90):
                endzoneDist_bins.append("> 90 yards")
        else:
            endzoneDist_bins.append("N/A")

    timeToThrow_bins = []
    for p in range(len(data['timeToThrow'])):
        if not np.isnan(data['timeToThrow'][p]):
            if data['timeToThrow'][p] < 2:
                timeToThrow_bins.append("< 2 seconds")
            elif (data['timeToThrow'][p] >= 2) and (data['timeToThrow'][p] < 5):
                timeToThrow_bins.append("2-5 seconds")
            elif (data['timeToThrow'][p] >= 5) and (data['timeToThrow'][p] < 10):
                timeToThrow_bins.append("5-10 seconds")
            elif (data['timeToThrow'][p] >= 10):
                timeToThrow_bins.append("> 10 seconds")
        else:
            timeToThrow_bins.append("No Pass")

    dropbackDist_bins = []
    for q in range(len(data['dropbackDistance'])):
        if not np.isnan(data['dropbackDistance'][q]):
            if (data['dropbackDistance'][q] >= 0) and (data['dropbackDistance'][q] < 3):
                dropbackDist_bins.append("0-3 yards")
            elif (data['dropbackDistance'][q] >= 3) and (data['dropbackDistance'][q] < 5):
                dropbackDist_bins.append("3-5 yards")
            elif (data['dropbackDistance'][q] >= 5) and (data['dropbackDistance'][q] < 8):
                dropbackDist_bins.append("5-8 yards")
            elif (data['dropbackDistance'][q] >= 8):
                dropbackDist_bins.append("> 8 yards")
        else:
            dropbackDist_bins.append("No Pass")

    minutes_remaining = []
    for r in range(len(data)):
        m = int(data['gameClock'][r].split(":")[0])
        if m < 2:
            minutes_remaining.append("< 2 minutes")
        elif (m >= 2) and (m < 4):
            minutes_remaining.append("2-4 minutes")
        elif (m >= 4) and (m < 6):
            minutes_remaining.append("4-6 minutes")
        elif (m >= 6) and (m < 8):
            minutes_remaining.append("6-8 minutes")
        elif (m >= 8) and (m < 10):
            minutes_remaining.append("8-10 minutes")
        elif (m >= 10) and (m < 12):
            minutes_remaining.append("10-12 minutes")
        elif (m >= 12):
            minutes_remaining.append("> 12 minutes")
    
    
    data['yardsGained_bins'] = yardsGained_bins
    data['passLen_bins'] = passLength_bins
    data['penaltyYards_bins'] = penaltyYards_bins
    data['endzoneDist_bins'] = endzoneDist_bins
    data['timeToThrow_bins'] = timeToThrow_bins
    data['dropBackDist_bins'] = dropbackDist_bins
    data['minutesRemaininginQuarter'] = minutes_remaining

    return data

In [6]:
games = pd.read_csv(f'{project_dir}/games.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
players = pd.read_csv(f'{project_dir}/players.csv')
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_1.csv')

motion2 = pd.read_csv("/kaggle/input/nfl-bdb25-build-dataset/motion2.csv")
motion2_control = pd.read_csv("/kaggle/input/nfl-bdb25-build-dataset/motion2_control.csv")
motion2_control2 = pd.read_csv("/kaggle/input/nfl-bdb25-build-dataset/motion2_control2.csv")

In [7]:
tmp = tmp.drop(['passDefensed','routeRan'], axis=1)

motion2 = bin_cont_cols(motion2)
motion2_control = bin_cont_cols(motion2_control)
motion2_control2 = bin_cont_cols(motion2_control2)

motion2 = motion2.merge(tmp, left_on=['game_id', 'playId'], right_on=['gameId','playId'])
motion2_control = motion2_control.merge(tmp, left_on=["gameId",'playId'], right_on=["gameId",'playId'])
motion2_control2 = motion2_control2.merge(tmp, left_on=["gameId", "playId"], right_on=["gameId", "playId"])

vals = ["C","I","S","R","IN"]
motion2['ispass'] = [0 if i not in vals else 1 for i in motion2['passResult']]
motion2_control['ispass'] = [0 if not i in vals else 1 for i in motion2_control['passResult']]
motion2_control2['ispass'] = [0 if not i in vals else 1 for i in motion2_control2['passResult']]

# Overview Visuals

In [8]:
gid = 2022091103
pid = 4300

fig = animate_play(games=games, tracking_df=tracking_df,
             play_df=plays, players=players, gameId=gid,
             playId=pid)

fig2 = go.Figure(fig)
fig2.show(renderer='iframe')

# Bar Charts

In [9]:
import plotly.express as px
import plotly.graph_objs as go

In [10]:
def make_route_df(data):
    percent_go = np.round(sum(data['GO']) / data.shape[0], 2) * 100
    percent_hitch = np.round(sum(data['HITCH']) / data.shape[0], 2) * 100
    percent_flat = np.round(sum(data['FLAT']) / data.shape[0], 2) * 100
    percent_out = np.round(sum(data['OUT']) / data.shape[0], 2) * 100
    percent_cross = np.round(sum(data['CROSS']) / data.shape[0], 2) * 100
    percent_in = np.round(sum(data['IN']) / data.shape[0], 2) * 100
    percent_post = np.round(sum(data['POST']) / data.shape[0], 2) * 100
    percent_slant = np.round(sum(data['SLANT']) / data.shape[0], 2) * 100
    percent_corner = np.round(sum(data['CORNER']) / data.shape[0], 2) * 100
    percent_screen = np.round(sum(data['SCREEN']) / data.shape[0], 2) * 100
    percent_angle = np.round(sum(data['ANGLE']) / data.shape[0], 2) * 100
    percent_wheel = np.round(sum(data['WHEEL']) / data.shape[0], 2) * 100
    
    cols = ['% Go', '% Hitch', '% Flat', '% Out', '% Cross', '% In','% Post', '% Slant','% Corner','% Screen', '% Angle', '% Wheel']
    
    out_df = pd.DataFrame(dict(values=[percent_go, 
                                       percent_hitch, 
                                       percent_flat, 
                                       percent_out,
                                       percent_cross,
                                       percent_in,
                                       percent_post,
                                       percent_slant,
                                       percent_corner,
                                       percent_screen,
                                       percent_angle,
                                       percent_wheel], columns=cols))

    return out_df
        

In [11]:
def pass_thrown_df(data):
    percent_thrown = np.round(sum(data['ispass']) / data.shape[0], 2) * 100
    cols = ['% Pass Play']

    out_df = pd.DataFrame(dict(values=[percent_thrown], columns=cols))

    return out_df

In [12]:
def passresult_df(data):
    catch_percent = np.round(data[data['passResult'] == 'C'].shape[0] / data.shape[0], 2) * 100
    incomplete_percent = np.round(data[data['passResult'] == 'I'].shape[0] / data.shape[0], 2) * 100
    sack_percent = np.round(data[data['passResult'] == 'S'].shape[0] / data.shape[0], 2) * 100
    scramble_percent = np.round(data[data['passResult'] == 'R'].shape[0] / data.shape[0], 2) * 100
    int_percent = np.round(data[data['passResult'] == 'IN'].shape[0] / data.shape[0], 2) * 100

    cols = ['Complete %', 'Incomplete %', 'Sack %', 'Scramble %', 'Interception %']

    out_df = pd.DataFrame(dict(values=[catch_percent,
                                       incomplete_percent,
                                       sack_percent,
                                       scramble_percent,
                                       int_percent], columns=cols))

    return out_df

In [13]:
ttt_df = make_route_df(motion2)
no_ttt_df = make_route_df(motion2_control)
no_ttt_df2 = make_route_df(motion2_control2)

In [14]:
fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=ttt_df['columns'], y=ttt_df['values'], text= np.round(ttt_df['values'],2)))
fig.add_trace(go.Bar(name='No TTT Shift', x=no_ttt_df['columns'], y=no_ttt_df['values'], text= np.round(no_ttt_df['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
#fig.update_layout(width=800, height=700)
fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))

fig.show(renderer='iframe')

In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=ttt_df['columns'], y=ttt_df['values'], text= np.round(ttt_df['values'],2)))
fig.add_trace(go.Bar(name='2x2 No Motion', x=no_ttt_df2['columns'], y=no_ttt_df2['values'], text= np.round(no_ttt_df2['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))

fig.show(renderer='iframe')

In [16]:
ttt_pass = pass_thrown_df(motion2)
no_ttt_pass = pass_thrown_df(motion2_control)

fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=ttt_pass['columns'], y=ttt_pass['values'], text= np.round(ttt_pass['values'],2)))
fig.add_trace(go.Bar(name='No TTT Shift', x=no_ttt_pass['columns'], y=no_ttt_pass['values'], text= np.round(no_ttt_pass['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
fig.update_layout(width=400, height=400)
fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))

fig.show(renderer='iframe')

In [17]:
ttt_pass = pass_thrown_df(motion2)
no_ttt_pass2 = pass_thrown_df(motion2_control2)

fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=ttt_pass['columns'], y=ttt_pass['values'], text= np.round(ttt_pass['values'],2)))
fig.add_trace(go.Bar(name='2x2 No Shift', x=no_ttt_pass2['columns'], y=no_ttt_pass2['values'], text= np.round(no_ttt_pass2['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
fig.update_layout(width=400, height=400)

fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))
fig.show(renderer='iframe')

In [18]:
passresult_ttt = passresult_df(motion2)
passresult_no_ttt = passresult_df(motion2_control)

fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=passresult_ttt['columns'], y=passresult_ttt['values'], text= np.round(passresult_ttt['values'],2)))
fig.add_trace(go.Bar(name='No TTT Shift', x=passresult_no_ttt['columns'], y=passresult_no_ttt['values'], text= np.round(passresult_no_ttt['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))

fig.show(renderer='iframe')

In [19]:
passresult_ttt = passresult_df(motion2)
passresult_no_ttt2 = passresult_df(motion2_control2)

fig = go.Figure()
fig.add_trace(go.Bar(name='TTT Shift', x=passresult_ttt['columns'], y=passresult_ttt['values'], text= np.round(passresult_ttt['values'],2)))
fig.add_trace(go.Bar(name='2x2 No Shift', x=passresult_no_ttt2['columns'], y=passresult_no_ttt2['values'], text= np.round(passresult_no_ttt2['values'],2)))

# Change the bar mode
fig.update_layout(barmode='group', template='plotly_white', legend=dict(orientation='h', x=0.3))
fig.update_layout(yaxis_title={'text': '% of Plays','font': {'size': 20}})
fig.update_traces(textfont_size=20)
fig.update_layout(xaxis = dict(tickfont = dict(size=17)))

fig.show(renderer='iframe')

# Funnel Graphs

In [20]:
def get_funnel_df(offenseForm, passCoverage, route, motion_data, control_data):
    m1 = motion_data['offenseFormation'] == offenseForm
    m2 = motion_data['pff_passCoverage'] == passCoverage
    m3 = motion_data['ispass'] == 1
    m4 = motion_data[route] == 1

    len1 = motion_data.loc[m1].shape[0]
    len2 = motion_data.loc[m1 & m2].shape[0]
    len3 = motion_data.loc[m1 & m2 & m3].shape[0]
    len4 = motion_data.loc[m1 & m2 * m3 & m4].shape[0]

    percent1 = np.round(len1 / motion_data.shape[0], 2) * 100
    percent2 = np.round(len2 / len1, 2) * 100
    percent3 = np.round(len3 / len2, 2) * 100
    percent4 = np.round(len4 / len3, 2) * 100

    stages = [offenseForm, passCoverage, 'Is Pass',route]
    motion = pd.DataFrame(dict(number=[percent1, percent2, percent3, percent4], stage=stages))
    motion['Motion'] = "TTT"

    m4 = control_data['offenseFormation'] == offenseForm
    m5 = control_data['pff_passCoverage'] == passCoverage
    m6 = control_data['ispass'] == 1
    m7 = control_data[route] == 1

    len4 = control_data.loc[m4].shape[0]
    len5 = control_data.loc[m4 & m5].shape[0]
    len6 = control_data.loc[m4 & m5 & m6].shape[0]
    len7 = control_data.loc[m4 & m5 & m6 & m7].shape[0]

    percent4 = np.round(len4 / control_data.shape[0], 2) * 100
    percent5 = np.round(len5 / len4, 2) * 100
    percent6 = np.round(len6 / len5, 2) * 100
    percent7 = np.round(len7 / len6, 2) * 100

    stages2 = [offenseForm, passCoverage,'Is Pass', route]
    no_motion = pd.DataFrame(dict(number=[percent4, percent5, percent6, percent7], stage=stages2))
    no_motion['Motion'] = "No TTT"

    funnel_df = pd.concat([motion, no_motion], axis=0)
    
    return funnel_df

In [21]:
def funnel_df_pass(motion_data, control_data, offenseForm, passCoverage):
    m1 = motion_data['offenseFormation'] == offenseForm
    m2 = motion_data['pff_passCoverage'] == passCoverage
    m3 = motion_data['ispass'] == 1

    len1 = motion_data.loc[m1].shape[0]
    len2 = motion_data.loc[m1 & m2].shape[0]
    len3 = motion_data.loc[m1 & m2 & m3].shape[0]

    percent1 = np.round(len1 / motion_data.shape[0], 2) * 100
    percent2 = np.round(len2 / len1, 2) * 100
    percent3 = np.round(len3 / len2, 2) * 100

    stages = [offenseForm, passCoverage, 'Is Pass']
    motion = pd.DataFrame(dict(number=[percent1, percent2, percent3], stage=stages))
    motion['Motion'] = "TTT"

    m4 = control_data['offenseFormation'] == offenseForm
    m5 = control_data['pff_passCoverage'] == passCoverage
    m6 = control_data['ispass'] == 1

    len4 = control_data.loc[m4].shape[0]
    len5 = control_data.loc[m4 & m5].shape[0]
    len6 = control_data.loc[m4 & m5 & m6].shape[0]

    percent4 = np.round(len4 / control_data.shape[0], 2) * 100
    percent5 = np.round(len5 / len4, 2) * 100
    percent6 = np.round(len6 / len5, 2) * 100

    stages2 = [offenseForm, passCoverage,'Is Pass']
    no_motion = pd.DataFrame(dict(number=[percent4, percent5, percent6], stage=stages2))
    no_motion['Motion'] = "No TTT"

    funnel_df = pd.concat([motion, no_motion], axis=0)
    
    return funnel_df

In [22]:
funnel_df1 = get_funnel_df('SINGLEBACK','Cover-1','CROSS',motion2,motion2_control)
funnel_df2 = get_funnel_df('SINGLEBACK', 'Cover-2', 'CROSS', motion2, motion2_control)
funnel_df3 = get_funnel_df('SINGLEBACK', 'Cover-3', 'CROSS', motion2, motion2_control)
funnel_df4 = get_funnel_df('SHOTGUN','Cover-1','CROSS',motion2,motion2_control)
funnel_df5 = get_funnel_df('SHOTGUN', 'Cover-2', 'CROSS', motion2, motion2_control)
funnel_df6 = get_funnel_df('SHOTGUN', 'Cover-3', 'CROSS', motion2, motion2_control)

funnel_df7 = get_funnel_df('SINGLEBACK','Cover-1','GO',motion2,motion2_control)
funnel_df8 = get_funnel_df('SINGLEBACK', 'Cover-2', 'GO', motion2, motion2_control)
funnel_df9 = get_funnel_df('SINGLEBACK', 'Cover-3', 'GO', motion2, motion2_control)
funnel_df10 = get_funnel_df('SHOTGUN','Cover-1','GO',motion2,motion2_control)
funnel_df11= get_funnel_df('SHOTGUN', 'Cover-2', 'GO', motion2, motion2_control)
funnel_df12 = get_funnel_df('SHOTGUN', 'Cover-3', 'GO', motion2, motion2_control)

In [23]:
funnel_df13 = get_funnel_df('SINGLEBACK','Cover-1','CROSS',motion2,motion2_control2)
funnel_df14 = get_funnel_df('SINGLEBACK', 'Cover-2', 'CROSS', motion2, motion2_control2)
funnel_df15 = get_funnel_df('SINGLEBACK', 'Cover-3', 'CROSS', motion2, motion2_control2)
funnel_df16 = get_funnel_df('SHOTGUN','Cover-1','CROSS',motion2,motion2_control2)
funnel_df17 = get_funnel_df('SHOTGUN', 'Cover-2', 'CROSS', motion2, motion2_control2)
funnel_df18 = get_funnel_df('SHOTGUN', 'Cover-3', 'CROSS', motion2, motion2_control2)

funnel_df19 = get_funnel_df('SINGLEBACK','Cover-1','GO',motion2,motion2_control2)
funnel_df20 = get_funnel_df('SINGLEBACK', 'Cover-2', 'GO', motion2, motion2_control2)
funnel_df21 = get_funnel_df('SINGLEBACK', 'Cover-3', 'GO', motion2, motion2_control2)
funnel_df22 = get_funnel_df('SHOTGUN','Cover-1','GO',motion2,motion2_control2)
funnel_df23= get_funnel_df('SHOTGUN', 'Cover-2', 'GO', motion2, motion2_control2)
funnel_df24 = get_funnel_df('SHOTGUN', 'Cover-3', 'GO', motion2, motion2_control2)

In [24]:
fig = px.funnel(funnel_df24, x='number', y='stage', color='Motion')
fig.update_layout(width=500, height=400)
fig.show(renderer='iframe')

In [25]:
pass_funnel1 = funnel_df_pass(motion2, motion2_control, 'SINGLEBACK','Cover-1')
pass_funnel2 = funnel_df_pass(motion2, motion2_control, 'SINGLEBACK','Cover-2')
pass_funnel3 = funnel_df_pass(motion2, motion2_control, 'SINGLEBACK','Cover-3')
pass_funnel4 = funnel_df_pass(motion2, motion2_control, 'SHOTGUN','Cover-1')
pass_funnel5 = funnel_df_pass(motion2, motion2_control, 'SHOTGUN','Cover-2')
pass_funnel6 = funnel_df_pass(motion2, motion2_control, 'SHOTGUN','Cover-3')

In [26]:
fig = px.funnel(pass_funnel6, x='number', y='stage', color='Motion')
fig.update_layout(width=500, height=400)
fig.show(renderer='iframe')

In [27]:
pass_funnel7 = funnel_df_pass(motion2, motion2_control2, 'SINGLEBACK','Cover-1')
pass_funnel8 = funnel_df_pass(motion2, motion2_control2, 'SINGLEBACK','Cover-2')
pass_funnel9 = funnel_df_pass(motion2, motion2_control2, 'SINGLEBACK','Cover-3')
pass_funnel10 = funnel_df_pass(motion2, motion2_control2, 'SHOTGUN','Cover-1')
pass_funnel11 = funnel_df_pass(motion2, motion2_control2, 'SHOTGUN','Cover-2')
pass_funnel12 = funnel_df_pass(motion2, motion2_control2, 'SHOTGUN','Cover-3')

In [28]:
fig = px.funnel(pass_funnel12, x='number', y='stage', color='Motion')
fig.update_layout(width=500, height=400)
fig.show(renderer='iframe')